In [55]:
import requests
from pyproj import Proj
import xml.etree.ElementTree as ET
import json
from datetime import datetime

# Introduction

According to their documentation, BRO | Basic Registration Subsurface still uses SOAP API for requesting data and only REST API if you provide data: \
*You can request fully automated BRO data (via a SOAP XML API). This is especially useful if you regularly need BRO data. Below we describe in 3 steps how you can connect to BRO issue API of Grondwatermonitoringnet (GMN) and what alternative options are to request BRO data.*

They further explain  
*The BRO's **SOAP APIs** are particularly suitable for organizations that want to frequently and fully automated **request** certain **BRO data** from a registration object. **You need a PKI certificate for this.** The SOAP APIs give direct access to the information as stored in the Basic Registration Ground. [...]. Certain data from a source document is also only visible to the source holder and data supplier, such as own references and Chamber of Commerce numbers. This also applies to data taken from registration. In the catalog of each registration object, it can be found which data is public or non-public in the explanation of the attribute. The models cannot be requested via the SOAP API'.*

**Request BRO data via SOAP web service** \
You can also request fully automated BRO data. This is especially useful if you want to request BRO data regularly. Read on the entation page of the SOAP web service what the possibilities are.

XML \
BRO data is delivered as XML. This makes it possible to link data from the BRO to other data.


Visualization of BRO data \
When loading BRO data into other systems, such as GIS/ArcGIS/QGis, you will not see any visualizations of the objects. These visualizations are not in the XML, so they are not shown. On BROloket you can of course view and save graphs and visualizations of BRO data. In addition, REST visualization services are available.


Do you want to view all measurements or research data from a specific area directly? Then go to the BROloket. You can consult all the information from the BRO data and BRO models directly on the map. BROloket also offers practical selection and filtering options in that viewer. You can interactively search for BRO data and view it in detail.

# Settings

In [2]:
url_openstreetmap = "https://nominatim.openstreetmap.org/search"

In [16]:
url_broservices = "https://publiek.broservices.nl/"

In [4]:
ENDPOINT_BROID = "/gm/gld/v1/waterlevel/extract/registration"
ENDPOINT_WATERLEVEL = "gm/gld/v1/waterlevel/extract/dispatch"

# From Address to broId

### From Address to Coordinates

Geocode the address to get lat/lon (using any geocoding service like Nominatim/OpenStreetMap or Google Maps).

In [5]:
user_input_address = "Depot Boijmans Van Beuningen"

In [6]:
geo = requests.get(
    url_openstreetmap, 
    headers={"User-Agent": "CaraLogic (contact: silvia@caralogic.com)"}, 
    params={"q": user_input_address, "format": "json", "limit": 1}
    )

geo.raise_for_status()
if len(geo.json()) == 0:
    print(f"no data found for {user_input_address}")
    
location = geo.json()[0]
latitude, longitude = location['lat'], location['lon']
print(f"Coordinates found for {user_input_address}: {latitude}, {longitude} (lat, lon)")

Coordinates found for Depot Boijmans Van Beuningen: 51.9139529, 4.4711320 (lat, lon)


#### Geometric Coordinates in RD Coordinates (Rijksdriehoek)

In [8]:
rd = Proj('epsg:28992')

x, y = rd(longitude, latitude)
x_m, y_m = x/1000, y/1000

print(f"RD coordinates in meter: x(east)={x_m}, y(north)={y_m}")

RD coordinates in meter: x(east)=91.9499085131148, y(north)=436.4531385572858


### From Coordinates to broId

To identify broIds for a specific address or coordinate

Basisregistratie Ondergrond (BRO)

# Query Data from broservices

In [19]:
bro_id = 'GTM0000000000102'
parameters = {'extractType': 'controlemeting'}

In [ ]:
def get_groundwater_levels_json(gld_bro_id):
    """
    Get groundwater level data in JSON format
    """
    url = f"https://publiek.broservices.nl/gm/gld/v1/objects/{gld_bro_id}"
    
    headers = {
        'Accept': 'application/json'
    }
    
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None


gld_id = "GLD000000002069"
response = get_groundwater_levels_json(gld_id)
response.text
#if data:
#    print(json.dumps(data, indent=2)[:2000])

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><dispatchDataResponse xmlns:gco="http://www.isotc211.org/2005/gco" xmlns:swe="http://www.opengis.net/swe/2.0" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:gldcommon="http://www.broservices.nl/xsd/gldcommon/1.0" xmlns:brocom="http://www.broservices.nl/xsd/brocommon/3.0" xmlns:gmd="http://www.isotc211.org/2005/gmd" xmlns:gml="http://www.opengis.net/gml/3.2" xmlns:om="http://www.opengis.net/om/2.0" xmlns:waterml="http://www.opengis.net/waterml/2.0" xmlns="http://www.broservices.nl/xsd/dsgld/1.0"><brocom:responseType>dispatch</brocom:responseType><brocom:requestReference>-</brocom:requestReference><brocom:dispatchTime>2025-10-22T22:55:19+02:00</brocom:dispatchTime><dispatchDocument><GLD_O gml:id="BRO_0008"><brocom:broId>GLD000000002069</brocom:broId><brocom:deliveryAccountableParty>30277172</brocom:deliveryAccountableParty><brocom:qualityRegime>IMBRO/A</brocom:qualityRegime><registrationHistory><brocom:objectRegistrationTime>2021-

In [57]:
def get_groundwater_levels(gld_bro_id):
    """
    Get groundwater level data from BRO and return as structured dict
    """
    url = f"https://publiek.broservices.nl/gm/gld/v1/objects/{gld_bro_id}"
    
    headers = {'Accept': 'application/xml'}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None
    
    return parse_gld_xml(response.text)

def parse_gld_xml(xml_text):
    """Parse GLD XML response into structured data"""
    
    # Define namespaces
    ns = {
        'gml': 'http://www.opengis.net/gml/3.2',
        'om': 'http://www.opengis.net/om/2.0',
        'waterml': 'http://www.opengis.net/waterml/2.0',
        'brocom': 'http://www.broservices.nl/xsd/brocommon/3.0',
        'gldcommon': 'http://www.broservices.nl/xsd/gldcommon/1.0',
        'swe': 'http://www.opengis.net/swe/2.0',
        'ns': 'http://www.broservices.nl/xsd/dsgld/1.0'
    }
    
    root = ET.fromstring(xml_text)
    
    # Extract metadata
    gld_data = {}
    
    # BRO ID
    bro_id = root.find('.//brocom:broId', ns)
    gld_data['gld_bro_id'] = bro_id.text if bro_id is not None else None
    
    # Research dates
    first_date = root.find('.//ns:researchFirstDate', ns)
    last_date = root.find('.//ns:researchLastDate', ns)
    gld_data['research_first_date'] = first_date.text if first_date is not None else None
    gld_data['research_last_date'] = last_date.text if last_date is not None else None
    
    # Monitoring well info
    gmw_id = root.find('.//gldcommon:GroundwaterMonitoringTube/gldcommon:broId', ns)
    tube_number = root.find('.//gldcommon:GroundwaterMonitoringTube/gldcommon:tubeNumber', ns)
    
    gld_data['gmw_bro_id'] = gmw_id.text if gmw_id is not None else None
    gld_data['tube_number'] = tube_number.text if tube_number is not None else None
    
    # Extract all observations
    gld_data['observations'] = []
    
    for observation in root.findall('.//om:OM_Observation', ns):
        obs_data = {}
        
        # Observation type
        obs_type = observation.find('.//om:value[@codeSpace="urn:bro:gld:ObservationType"]', ns)
        obs_data['observation_type'] = obs_type.text if obs_type is not None else None
        
        # Time period
        begin_pos = observation.find('.//gml:beginPosition', ns)
        end_pos = observation.find('.//gml:endPosition', ns)
        obs_data['period_start'] = begin_pos.text if begin_pos is not None else None
        obs_data['period_end'] = end_pos.text if end_pos is not None else None
        
        # Extract measurement points
        obs_data['measurements'] = []
        
        for point in observation.findall('.//waterml:MeasurementTVP', ns):
            measurement = {}
            
            # Time
            time_elem = point.find('waterml:time', ns)
            measurement['time'] = time_elem.text if time_elem is not None else None
            
            # Value
            value_elem = point.find('waterml:value', ns)
            if value_elem is not None:
                is_nil = value_elem.get('{http://www.w3.org/2001/XMLSchema-instance}nil')
                if is_nil == 'true':
                    measurement['value'] = None
                else:
                    measurement['value'] = float(value_elem.text)
                    measurement['unit'] = value_elem.get('uom', 'm')
            
            # Quality
            quality_elem = point.find('.//swe:value', ns)
            measurement['quality'] = quality_elem.text if quality_elem is not None else None
            
            obs_data['measurements'].append(measurement)
        
        gld_data['observations'].append(obs_data)
    
    return gld_data

In [ ]:
gld_id = "GLD000000002069"
bro_id

In [ ]:
data = get_groundwater_levels(gld_id)

if data:
    print(f"GLD ID: {data['gld_bro_id']}")
    print(f"GMW ID: {data['gmw_bro_id']}")
    print(f"Tube Number: {data['tube_number']}")
    print(f"Research Period: {data['research_first_date']} to {data['research_last_date']}")
    print(f"\nNumber of observation series: {len(data['observations'])}")
    
    # Show first few measurements from first observation
    if data['observations']:
        first_obs = data['observations'][0]
        print(f"\nObservation Type: {first_obs['observation_type']}")
        print(f"Period: {first_obs['period_start']} to {first_obs['period_end']}")
        print(f"Total measurements: {len(first_obs['measurements'])}")
        print("\nFirst 5 measurements:")
        for m in first_obs['measurements'][:5]:
            print(f"  {m['time']}: {m['value']} {m.get('unit', 'm')} (quality: {m['quality']})")
    
    # Save to JSON file
    with open('groundwater_data.json', 'w') as f:
        json.dump(data, f, indent=2)
    print("\nData saved to groundwater_data.json")

In [ ]:
url = "https://service.pdok.nl/bzk/brogtm/atom/v1_1/index.xml"

save_geotop_data = "geotop_data.xml"
save_geotop_atom = "geotop_atom.xml"

In [ ]:
response = requests.get(url)
response.raise_for_status()

In [ ]:
xml_content = response.content
root = ET.fromstring(xml_content)

{http://www.w3.org/2005/Atom}feed


### Save actual data as XML

In [ ]:
namespaces = {'atom': root.tag.split('}')[0].split('{')[1]}

for entry in root.findall('atom:entry', namespaces):
    bro_id = entry.find('atom:id', namespaces).text
    title = entry.find('atom:title', namespaces).text
    updated = entry.find('atom:updated', namespaces).text
    print(f"BRO ID: {bro_id}, Title: {title}, Updated: {updated}")


BRO ID: https://service.pdok.nl/bzk/brogtm/atom/v1_1/brogtm.xml, Title: BRO GeoTOP (GTM), Updated: 2025-03-18T10:56:30Z


In [ ]:
data_url = entry.find('atom:link', namespaces).attrib['href']

data_response = requests.get(data_url)
data_response.raise_for_status()

with open(save_geotop_data, "wb") as f:
    f.write(data_response.content)

### Open Data

In [168]:
tree = ET.parse("geotop_data.xml")
root = tree.getroot()

# Check root tag
print("Root tag:", root.tag)

Root tag: {http://www.opengis.net/cat/csw/2.0.2}GetRecordByIdResponse


In [192]:
namespaces = {
    'gmd': 'http://www.isotc211.org/2005/gmd',
    'gmx': 'http://www.isotc211.org/2005/gmx',
    'xlink': 'http://www.w3.org/1999/xlink'
}

In [200]:
url_elements = root.findall(".//gmd:URL", namespaces)
urls = [elem.text for elem in url_elements]

url_atom = None
for url in urls:
    if 'atom' in url:
        url_atom = url
        
url_atom

'https://service.pdok.nl/bzk/brogtm/atom/v1_1/brogtm.xml'

In [ ]:
response = requests.get(url_atom)

response.raise_for_status()
with open(save_geotop_atom, "wb") as f:
    f.write(response.content)

In [ ]:
atom_file = "geotop_atom.xml"

# Parse the XML
tree = ET.parse(atom_file)
root = tree.getroot()

# Atom namespace
namespaces = {'atom': 'http://www.w3.org/2005/Atom'}


In [205]:
# Loop through all entries
for entry in root.findall('atom:entry', namespaces):
    bro_id_url = entry.find('atom:id', namespaces).text
    title = entry.find('atom:title', namespaces).text
    updated = entry.find('atom:updated', namespaces).text
    
    # Some entries have a link to the actual dataset
    link_elem = entry.find('atom:link', namespaces)
    data_link = link_elem.attrib['href'] if link_elem is not None else "No link"
    
    print(f"BRO ID: {bro_id_url}")
    print(f"Title: {title}")
    print(f"Updated: {updated}")
    print(f"Data link: {data_link}")
    print("-" * 50)


BRO ID: https://service.pdok.nl/bzk/brogtm/atom/v1_1/brogtm.xml
Title: BRO GeoTOP (GTM)
Updated: 2025-03-18T10:56:30Z
Data link: https://service.pdok.nl/bzk/brogtm/atom/v1_1/downloads/brogtm.zip
--------------------------------------------------


In [207]:
atom_file = "geotop_atom.xml"
tree = ET.parse(atom_file)
root = tree.getroot()

namespaces = {
    'atom': 'http://www.w3.org/2005/Atom'
}

# Find the <link> element with type="application/zip"
zip_link = None
for link in root.findall('atom:entry/atom:link', namespaces):
    if link.attrib.get('type') == 'application/zip':
        zip_link = link.attrib['href']
        break

print("Download URL for GeoTOP dataset ZIP:", zip_link)


Download URL for GeoTOP dataset ZIP: https://service.pdok.nl/bzk/brogtm/atom/v1_1/downloads/brogtm.zip


In [ ]:
response = requests.get(zip_link, stream=True)
with open("brogtm.zip", "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print("Downloaded brogtm.zip")


In [ ]:
with zipfile.ZipFile("brogtm.zip", 'r') as zip_ref:
    zip_ref.extractall("brogtm_data")

print("Extracted files:", zip_ref.namelist())
